## Logging HowTo
### 基本的Loggin
Logging用于追踪事件.事件的重要性成为level或severity.
### 何时使用logging
logging提供了一系列方便的方法用于简单的logging. 分别是: debug(), info(), warning(), error(), critical(), 下表描述了何种情况下的最佳使用方法.


想要执行的任务 | 处理该任务的最佳方法
-------------- | --------------------
编程时或脚本时显示控制台输出 | print()
报告程序正常运行时的消息 | logging.info() (或者logging.debug()处于诊断的目的用于具体的显示细节)
发出在特定运行时的warning | 如果事件是可以避免的,并且服务端应该做出修改来消除该warning的, 用warnings.warn(); 如果服务端没有办法处理的,但要指出该warning, 使用loggign.warning()
报告运行期间的一个特定error | Raise an exception
在没有raise exception的情况下报告error | logging.error(), logging.exception() or logging.critical()
<br \>
logging函数是根据level(severity)来命名的:

level | 使用场合
----- | ---------
DEBUG | 细节信息, 用于诊断程序
INGFO | 说明程序按预期执行
WARNING | 出乎意料的事情发生了; 或者未来将会发生的问题
ERROR | 因为一些严重的问题, software不能正常运行某些功能
CRITICAL | 严重问题, 显示program本身可能不能继续运行

----------------

### 例子

In [1]:
import logging
logging.warning('Watch out!')
logging.info("I told you so.")

INFO消息没有打印是因为默认的level是WARNING. 打印出来的消息包括level和具体消息. root暂时不用管是什么, 下面会讲到.

### 输出log到文件
这里把log阈值设置为DEBUG, 所以level(severity)大于或等于debug的log都会记录到. 参数level也可以是数字
<br \>
(下面的命令在jupyter里不能输出到文件, 在脚本中可以)

In [3]:
import logging
logging.basicConfig(filename='example.log', level=logging.DEBUG)
logging.debug("This message should go to the log file.")
logging.info("And so should this.")
logging.warning("And this")

**logging.basicConfig的调用要在logging.debug(), logging.warning()等的调用之前(well, obviously), 反复调用以上脚本, 后面的消息会追加到example.log中, 如果希望每次运行清掉以前的log, 可以用:**<br \>
```logging.basicConfig(filename='example.log', filemode='w', level=logging.DEBUG)```

### 多模块(文件)log
如果程序由多个模块(文件)组成, 下面是一个组织log的例子:

```Python
# myapp.py
import logging
import mylib

def main():
    logging.basicConfig(filename='myapp.log', level=logging.INFO)
    logging.info('Started')
    mylib.do_something()
    logging.info('Finished')

if __name__ == '__main__':
    main()
 ```
 <br\>
 
 ```Python
# mylib.py
import logging

def do_something():
    logging.info('Doing something')
```

运行myapp.py, 结果是
```Python
INFO:root:Started
INFO:root:Doing something
INFO:root:Finished
```

**但是这样有一个缺点: 只能看到log的消息, 不能看到log来源于哪个模块, 处于程序的哪个部位. 要追踪消息的位置, 需要看高级日志记录教程.**

### 格式化消息
log也支持format格式化, 但文档用的是%s, 说是为了向后兼容, 这里往后再看.

[在整个应用程序中使用特定格式化样式](http://python.usyiyi.cn/documents/python_352/howto/logging-cookbook.html#formatting-styles)

### 更改显示消息的格式
```Python
import logging
logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.DEBUG)
logging.debug('This message should appear on the console')
logging.info('So should this')
logging.warning('And this, too')
```
将会打印
```Python
DEBUG:This message should appear on the console
INFO:So should this
WARNING:And this, too
```
**注意到消息中的'root'不见了, 更多可用的格式化字符串参考**
    [LogRecord attributes](http://python.usyiyi.cn/documents/python_352/library/logging.html#logrecord-attributes)
    

## 高级日志教程
logging提供了模块化设计, 提供了Logger, Handler, Filter, Formatter
- Logger 暴露了应用程序中可以直接使用的接口.
- Handler 决定把(Logger产生的)日志发送至合适的目的地.
- Filter 提供更好的粒度控制(grained facility), 它可以决定输出哪些日志记录.
- Formatter 指明了最终输出的日志的布局.

Logging都是调用Logger类的实例方法实现的. 他们在概念上组织成层级式的命名空间, 它们通过'.'进行分割, 例如: 假设有一个logger叫做'scan', 则它是'scan.html', 'scan.pdf'的父logger. 一个好的方法是使用模块的名字作为logger的名字,这样就可以通过包/模块的方式来追踪日志.
```Python
logger = logging.getlogger(__name__)
```
basicConfig设置的默认格式是```severity:logger name:message```, 而所有logger的根层级(root hierarchy)的名字都是root, 所以上面打印的日志有个'root', 默认的info(), debug(), warning(), error(), critical()都是调用root logger.
<br\><br\>
**logger的工作流程如图所示[logger流程](https://docs.python.org/3/_images/logging_flow.png)**